# GraphMAE 
#intro 以mask rate=0.5进行实验，num_mask_nodes = int(0.5 * num_nodes) 保持可见的节点数量一致  Meta_Self-0.25

######## distribution masking 
num_mask_nodes = int(mask_rate * num_nodes)
idx_val   = g.ndata['val_mask'].nonzero().squeeze()
idx_test  = g.ndata['test_mask'].nonzero().squeeze()
idx_train = g.ndata['train_mask'].nonzero().squeeze()

idx_mask = torch.cat((idx_test, idx_val))
idx_keep = idx_train  # 可以设计实验不断的减小有效信息的可见范围

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()
num_mask_nodes = len(mask_nodes)


#Cora 
test->train: torch.cat((idx_test, idx_val)) -> idx_train   30.18%
train->test: idx_train -> torch.cat((idx_test, idx_val))   62.58%

#Citeseer
test->train: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train   49.35%
train->test: idx_train -> torch.cat((idx_test, idx_val))                    63.98%

#CoraML
test->train: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train   27.80%
train->test: idx_train -> torch.cat((idx_test, idx_val))[:num_mask_nodes]   57.83%


Strategy                             & Attacked-Cora       &Attacked-Citeseer   & Attacked-CoraML
test->train                          & 30.18%              &49.35%              &27.80%   \\
random                               & 48.74%              &62.50%              &41.50%   \\
train->test                          & 62.58%              &63.98%              &57.83%   \\

#######################################################
#######################################################
#######################################################
#######################################################
#######################################################

# 不同的训练分布，发现只要虽然train的比例高达90%，但只要训练的epoch足够多，错误的引导一定会误导encoder
# 这里利用3000个epoch做实验 
# 比较对象为 
# train/val/test 比例为0.9/0.05/0.05 


#Cora 0.9/0.05/0.05的情况主要攻击test，但train的比例太高了，所以攻击有限
test->train: idx_test  -> torch.cat((idx_train, idx_val))
train->test: torch.cat((idx_train, idx_val)) -> idx_test 



# 在2000个epoch之后才逐渐开始越来越差  train/val/test 0.9/0.05/0.05
Strategy                             & Attacked-Cora      
test->train                          & 74.40           \\
random                               & 73.60           \\

train->test                          & 55.20%          \\ # 除了seed 1，其余seed都会衰减
# 补充一个同比例的实验，利用剩下的重建0.05的train，发现并不会误导encoder的学习
train->test                          & 74.40%          \\


# GraphMAE2
#intro 以mask rate=0.5进行实验，num_mask_nodes = int(0.5 * num_nodes) 保持可见的节点数量一致

######## distribution masking (add ssh)
num_mask_nodes = int(mask_rate * num_nodes)
idx_val   = g.ndata['val_mask'].nonzero().squeeze()
idx_test  = g.ndata['test_mask'].nonzero().squeeze()
idx_train = g.ndata['train_mask'].nonzero().squeeze()
# 确定keep和mask的分布范围
idx_keep_dis = torch.cat((idx_test, idx_val))
idx_mask_dis = idx_train  # 可以设计实验不断的减小有效信息的可见范围
# 在各自的分布范围内进行随机选择
perm_keep = torch.randperm(len(idx_keep_dis), device=x.device)
perm_mask = torch.randperm(len(idx_mask_dis), device=x.device)
# 挑选keep和mask的节点 #仅修改这里perm_keep的范围即可，例如perm_keep[:num_mask_nodes]
idx_keep = idx_keep_dis[perm_keep[:num_mask_nodes]]
idx_mask = idx_mask_dis  

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()



# Cora 0.25
test->train /fixed  remask: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train  31.89%
test->      /random remask: torch.cat((idx_test, idx_val))[:num_mask_nodes]               57.29%

train->     /random remask: idx_train->                                                   55.68%
train->test /fixed  remask: idx_train->torch.cat((idx_test, idx_val))                     58.10%



# Citeseer 0.25
test->train /fixed  remask: torch.cat((idx_test, idx_val))[:num_mask_nodes] -> idx_train  49.53
test->      /random remask: torch.cat((idx_test, idx_val))[:num_mask_nodes]               51.84  #这里用mask=0.1，可以说的通，可见范围不变，只是所用的比例不同，要不mask=0.5的话数据差距有点大

train->     /random remask: idx_train->                                                   65.64
train->test /fixed  remask: idx_train->torch.cat((idx_test, idx_val))[:num_mask_nodes]    66.65           

seed 1/2/3


Strategy                        & Attacked-Cora                  & Attacked-Citeseer            & DScore
test->train /fixed  remask      & 31.89%/31.29%/32.39%           & 49.53%/49.29%/49.06%         & 1.0
test->      /random remask      & 51.81%/52.36%/50.15%           & 57.29%/58.95%/59.60%         & 0.1

random      /random remask      & 45.22%/45.32%/44.25%           & 64.45%/64.16%/62.44%         & 0.54
random      /fixed  remask      & 47.89%/46.38%/47.43%           & 64.45%/63.68%/61.14%         & 0.54

train->     /random  remask     & 55.68%/57.24%/56.14%           & 65.64%/64.75%/64.81%         & 0.002
train->test /fixed   remask     & 58.10%/56.34%/57.49%           & 66.65%/66.05%/66.00%         & 0.0

# node center cosine similarity 分析
# 对于Meta_Self cora 0.25  train/val/test 0.1/0.1/0.8
攻击之后的train node  91605
攻击之前的train node  310014

# 实验现象发现 #1-4 四个参数 
让keep的可见范围大一些，但每个epoch取的比例小一点以增加随机性， mask范围小一些，即把目光更聚集在后面的节点上，mask的比例大一些，尽量把这些大的节点全都重建，这样的效果要好很多，比如keep可见前900个，每个epoch取的比例为0.4，mask只取排序2200以后的节点，即最后200个具有最好的sim的节点，每个epoch的重建比例为0.8，这样的训练效果能在0.5719


######### node center cosine similarity masking
idx_keep_dis = g.ndata['csim_sorted'][:900]                          #1
idx_mask_dis = g.ndata['csim_sorted'][2200:]                         #2
perm_keep  = torch.randperm(len(idx_keep_dis), device=x.device)
perm_mask  = torch.randperm(len(idx_mask_dis), device=x.device)

num_dis_keep_nodes = int(len(idx_keep_dis) * 0.4)                    #3
num_dis_mask_nodes = int(len(idx_mask_dis) * 0.8)                    #4
idx_keep   = idx_keep_dis[perm_keep[:num_dis_keep_nodes]]
idx_mask   = idx_mask_dis[perm_mask[:num_dis_mask_nodes]]  

keep_nodes = idx_keep.cuda()
mask_nodes = idx_mask.cuda()
num_mask_nodes = len(mask_nodes)


GraphMAE                                       (很奇怪啊，攻击比例大于0.1反而效果更好了，这方法有问题的感觉)
polblogs meta      DICE				random          heuristic
0.9476±0.0036	   0.9476±0.0036   0.9476±0.0036    0.0   95.70(gcn)   0.9476±0.0036(GraphMAE)
0.8811±0.0275	   0.8592±0.0026   0.9156±0.0049    0.05  78.52(gcn)   0.7843±0.0589(GraphMAE)
0.8142±0.0570	   0.7922±0.0017   0.9095±0.0026    0.1   74.43(gcn)   0.7436±0.0329(GraphMAE)
0.7014±0.0093	   0.7354±0.0023   0.8924±0.0023
0.6777±0.0099	   0.6897±0.0017   0.8865±0.0014
0.6667±0.0067	   0.6414±0.0015   0.8735±0.0020

和鲁棒性方法比较的参数设置 在预训练过程中使用不同的鲁棒方法，但是在下游任务上依然对原始图进行预测 seed 1 
jaccard运行比较快，Guard和SVD超级慢,而且svd太差了，基本就是失效状态

GraphMAE+GATGuard  编码器采用GATGuard 解码器依然沿用GAT
GraphMAE+Jaccard   Cora   threshold 0.06 当不攻击的时候threshold 0.0
                   CoraML 0.25 threshold 0.1   
                          0.15 threshold 0.06  
                          当不攻击的时候threshold 0.04 稍加修剪效果可以85-86以上
GraphMAE+svd       k=50

ogbn-arxiv 实验
full              全图训练发现num hiddem小一些效果更好 所以用512
Ptb Rate 50\%     DICE   	        random          PRBCD           Heuristic     
GraphMAE          56.53             63.08           60.52           53.59
GraphMAE2         53.56             57.10           54.45           50.65
FilterMGAE        56.32             63.20           60.82           59.21


mini batch        minibatch还是num hiddem = 1024要好一些(用小一些的num_hidden变化不大) batch_size: 512  batch_size_f: 256
Ptb Rate 50\%     DICE   		    random   		PRBCD   	 	Heuristic
GraphMAE          59.92(\%)  		65.37(\%)      	62.29(\%)   	53.00(\%)       
GraphMAE2         57.56(\%)  		63.41(\%)     	61.72(\%)   	52.56(\%)

各种GAE实验数据记录及重要参数记录
GAE                         out_channels
#######################################################
Cora        Meta      0.25      16         49.56 ± 1.88
                                32         45.48 ± 0.68
                                64         49.01 ± 0.94

            DICE      0.5       16         56.77 ± 0.51 
                                32         59.72 ± 0.91
                                64         62.25 ± 0.28 

            random    0.5       16         62.66 ± 1.22
                                32         65.76 ± 0.54
                                64         66.43 ± 0.43

            heuristic 0.5       16         59.85 ± 0.86
                                32         61.59 ± 0.70
                                64         55.96 ± 0.67
#######################################################
CoraML      Meta      0.25      16         41.75 ± 1.41
                                32         45.63 ± 0.80
                                64         41.90 ± 1.20

            DICE      0.5       16         50.95 ± 0.86
                                32         55.94 ± 1.47
                                64         63.00 ± 0.81

            random    0.5       16         58.47 ± 0.29
                                32         67.18 ± 0.54
                                64         62.42 ± 0.38

            heuristic 0.5       16         52.07 ± 0.92
                                32         55.06 ± 0.86
                                64         62.22 ± 0.25
#######################################################
Citeseer    Meta      0.25      16         53.21 ± 2.33
                                32         52.61 ± 1.23
                                64         53.28 ± 0.89

            DICE      0.5       16         50.04 ± 1.03
                                32         52.75 ± 0.82
                                64         57.08 ± 0.59

            random    0.5       16         56.71 ± 1.40
                                32         55.42 ± 0.58
                                64         60.47 ± 0.40

            heuristic 0.5       16         51.32 ± 1.95
                                32         49.45 ± 0.43
                                64         58.66 ± 0.49
#######################################################
Pubmed      Meta      0.25      16         58.35 ± 0.47
                                32         66.62 ± 0.14
                                64         65.53 ± 0.08

            DICE      0.5       16         62.26 ± 0.27
                                32         65.66 ± 0.20
                                64         65.85 ± 0.11   

            random    0.5       16         73.27 ± 0.07
                                32         74.89 ± 0.05
                                64         75.42 ± 0.03

            heuristic 0.5       16         74.13 ± 1.79
                                32         78.51 ± 0.60
                                64         78.15 ± 0.42
#######################################################
Polblogs    Meta      0.25      16         68.91 ± 8.39   #波动非常大
                                32         64.33 ± 7.32
                                64         66.68 ± 2.56

            DICE      0.5       16         71.22 ± 1.58
                                32         69.47 ± 1.61
                                64         71.83 ± 1.09

            random    0.5       16         86.40 ± 1.63
                                32         88.60 ± 0.80
                                64         88.50 ± 0.62

            heuristic 0.1       16         61.55 ± 7.67 
                                32         66.67 ± 6.62
                                64         65.32 ± 6.69

In [ ]:
VGAE        
#######################################################   64, 128比较好，大于128没有什么变化
Cora        Meta      0.25      16         43.94 ± 1.82      
                                32         43.96 ± 2.30
                                64         47.34 ± 2.32
                                128        51.75 ± 2.42

            DICE      0.5       16         55.37 ± 0.54
                                32         54.75 ± 0.32
                                64         65.15 ± 0.18
                                128        62.96 ± 0.54

            random    0.5       16         60.17 ± 1.11
                                32         66.75 ± 0.72
                                64         67.63 ± 0.74
                                128        71.98 ± 0.24

            heuristic 0.5       16         45.51 ± 1.65
                                32         52.52 ± 1.07
                                64         61.54 ± 1.05
                                128        65.13 ± 0.13
#######################################################
CoraML      Meta      0.25      16         46.41 ± 2.79 
                                32         44.31 ± 1.36
                                64         46.25 ± 4.51
                                128        44.07 ± 1.92

            DICE      0.5       16         46.63 ± 0.83
                                32         48.46 ± 0.49
                                64         60.77 ± 0.83
                                128        63.06 ± 1.07

            random    0.5       16         51.35 ± 0.66
                                32         62.98 ± 0.74
                                64         67.69 ± 0.37
                                128        72.66 ± 0.23

            heuristic 0.5       16         36.41 ± 1.67
                                32         48.87 ± 1.02
                                64         63.56 ± 0.28
                                128        67.08 ± 0.20
#######################################################
Citeseer    Meta      0.25      16         61.48 ± 1.46
                                32         57.39 ± 1.96
                                64         55.58 ± 1.98
                                128        51.00 ± 1.70    

            DICE      0.5       16         45.27 ± 1.00
                                32         49.56 ± 0.95
                                64         53.02 ± 0.28
                                128        50.66 ± 0.51

            random    0.5       16         52.06 ± 1.36
                                32         58.19 ± 1.02
                                64         58.60 ± 0.26
                                128        55.63 ± 0.78

            heuristic 0.5       16         53.46 ± 1.54 
                                32         58.96 ± 0.83
                                64         57.82 ± 0.49
                                128        60.63 ± 0.25 
#######################################################  1024 更大一些更好，为了比较采用512或1024，2048反而衰减严重
Pubmed      Meta      0.25      16         52.92 ± 0.27
                                32         55.03 ± 0.07
                                64         58.03 ± 0.08
                                128        63.96 ± 0.19
                                256        65.48 ± 0.04
                                512        62.06 ± 0.11
                                1024       68.11 ± 0.09
                                2048       57.21 ± 0.17

            DICE      0.5       16         61.76 ± 0.21
                                32         63.57 ± 0.09
                                64         63.42 ± 0.31
                                128        66.15 ± 0.02
                                256        66.58 ± 0.03
                                512        65.59 ± 0.07
                                1024       65.54 ± 0.12

            random    0.5       16         68.54 ± 0.07
                                32         70.47 ± 0.03
                                64         71.68 ± 0.05
                                128        73.78 ± 0.05
                                256        74.35 ± 0.03
                                512        75.06 ± 0.09
                                1024       72.47 ± 0.13


            heuristic 0.5       16         61.88 ± 1.41
                                32         70.77 ± 0.52
                                64         75.30 ± 0.32
                                128        76.20 ± 0.20
                                256        77.19 ± 0.17
                                512        78.17 ± 0.23
                                1024       77.79 ± 0.31
#######################################################
Polblogs    Meta      0.25      16         70.95 ± 7.79
                                32         61.56 ± 0.96
                                64         69.84 ± 10.48
                                128        63.51 ± 0.59

            DICE      0.5       16         71.21 ± 0.28
                                32         71.56 ± 0.37
                                64         70.44 ± 0.71
                                128        70.49 ± 1.00

            random    0.5       16         87.74 ± 1.17
                                32         88.32 ± 0.61
                                64         88.01 ± 0.94
                                128        87.98 ± 1.26

            heuristic 0.1       16         71.21 ± 9.42
                                32         62.82 ± 12.38
                                64         64.88 ± 11.85
                                128        54.30 ± 6.55


In [ ]:
S2GAE       攻击情况下hidden小一些会好,默认128，但pubmed这样的大图上还是大一点好
####################################################### hidden 128 batch 1024 64不如128
Cora        Meta      0.25         51.35 ± 0.52
            DICE      0.5          56.00 ± 0.79
            random    0.5          63.02 ± 0.44
            heuristic 0.5          48.51 ± 0.10
####################################################### hidden 128 batch 1024 64不如128
CoraML      Meta      0.25         36.92 ± 0.45
            DICE      0.5          56.46 ± 0.40
            random    0.5          64.01 ± 0.51
            heuristic 0.5          48.13 ± 1.36
####################################################### hidden 64  batch 1024
Citeseer    Meta      0.25         57.18 ± 0.10
            DICE      0.5          47.82 ± 0.85
            random    0.5          57.62 ± 0.64
            heuristic 0.5          54.62 ± 2.25
####################################################### hidden 128 batch 8192
Pubmed      Meta      0.25         75.06 ± 0.74
            DICE      0.5          67.39 ± 0.27 
            random    0.5          79.52 ± 0.61
            heuristic 0.5          73.32 ± 0.66
####################################################### hidden 64  batch 1024
Polblogs    Meta      0.25         65.03 ± 2.82
            DICE      0.5          70.24 ± 0.79
            random    0.5          86.63 ± 0.71
            heuristic 0.1          62.95 ± 2.75         

MaskGAE
####################################################### Mask Path  encoder_channels 128 full_data
Cora        Meta      0.25        53.36 ± 0.96 
            DICE      0.5         68.78 ± 0.44 
            random    0.5         74.09 ± 0.69 
            heuristic 0.5         63.77 ± 0.32
####################################################### Mask Edge  encoder_channels 64  full_data
CoraML      Meta      0.25        38.79 ± 1.67
            DICE      0.5         68.69 ± 0.20
            random    0.5         75.84 ± 0.48 
            heuristic 0.5         60.99 ± 0.88
####################################################### Mask Path  encoder_channels 64  full_data
Citeseer    Meta      0.25        62.16 ± 0.58 
            DICE      0.5         47.50 ± 0.53 
            random    0.5         53.89 ± 1.63 
            heuristic 0.5         47.46 ± 2.39 
####################################################### Mask Path encoder_channels 128 full_data nodeclas_weight_decay 1e-3 run 5很关键，这个值影响太大
Pubmed      Meta      0.25        76.03 ± 0.04 
            DICE      0.5         67.40 ± 0.02 
            random    0.5         76.56 ± 0.16 
            heuristic 0.5         63.59 ± 0.10 
####################################################### Mask Edge  encoder_channels 128 full_data run 5
Polblogs    Meta      0.25        59.30 ± 11.10  
            DICE      0.5         66.46 ± 0.43 
            random    0.5         86.85 ± 0.95 
            heuristic 0.1         59.04 ± 13.73 

GraphMAE2 A800   这里感觉可以在论文里提一下，在攻击的情况下就是刚开始的时候效果很好，但epoch增大越来越下降，所以没有防御能力, 感觉不要钻牛角尖，比较的时候条件相同，主要看random和fixed就行，所有参数都调累死
#######################################################     num_hidden: 512    1500 epoch
Cora        Meta      0.25        54.28 ± 0.95              mask_rate 0.8 fixed         random引入了更多的随机性
            DICE      0.5         66.30 ± 0.37              mask_rate 0.5 random
            random    0.5         76.96 ± 0.30              mask_rate 0.5 fixed
            heuristic 0.5         71.43 ± 1.28              mask_rate 0.8 fixed

#######################################################     num_hidden: 512    1500 epoch
CoraML      Meta      0.25        49.11 ± 0.22              mask_rate 0.8 random        这个反而random效果比较好，很有趣
            DICE      0.5         72.33 ± 0.45              mask_rate 0.8 random        在重建的时候random比fixed要好太多了，是不是在重建的时候引入更多的随机性对鲁棒性要更好？
            random    0.5         78.65 ± 0.53              mask_rate 0.5 random        感觉graphmae2的鲁棒性效果要好很多
            heuristic 0.5         72.39 ± 0.22              mask_rate 0.8 random        前200个epoch效果好到惊人，但是随着训练推进，下降非常明显, random要好于fixed, 所以取个1000 epoch的中间值吧

#######################################################     num_hidden: 64
Citeseer    Meta      0.25        64.07 ± 0.91              mask_rate 0.5 random
            DICE      0.5         63.09 ± 0.28              mask_rate 0.5 random
            random    0.5         67.12 ± 0.36              mask_rate 0.5 random
            heuristic 0.5         68.06 ± 0.33              mask_rate 0.8 random
            
#######################################################     num_hidden: 1024    2000 epoch
Pubmed      Meta      0.25        77.21 ± 0.86              mask_rate 0.5 random
            DICE      0.5         69.27 ± 1.34              mask_rate 0.5 random
            random    0.5         78.20 ± 0.78              mask_rate 0.5 random
            heuristic 0.5         83.42 ± 1.46              mask_rate 0.5 random

#######################################################     num_hidden: 64    1500 epoch
Polblogs    Meta      0.25       54.32 ± 5.74               mask_rate 0.5 random
            DICE      0.5        69.76 ± 0.58               mask_rate 0.5 random 
            random    0.5        85.81 ± 0.55               mask_rate 0.5 random
            heuristic 0.1        57.92 ± 4.61               mask_rate 0.5 random